In [4]:
import os
import pandas as pd
import dask.bag as db
from dask.distributed import Client

# Define the directory containing the CSV files
directory = r'D:\SamsungSTF\Processed_Data\TripByTrip'

# Function to check if a file contains a period where speed is 0 for 5 minutes or more
def check_file(filepath):
    try:
        df = pd.read_csv(filepath)
        if 'speed' in df.columns and 'TripTime' in df.columns:
            # Ensure TripTime is sorted
            df = df.sort_values(by='TripTime')
            
            # Filter rows where speed is 0
            zero_speed_df = df[df['speed'] == 0]
            
            if not zero_speed_df.empty:
                # Calculate the duration of zero speed periods
                zero_speed_df['time_diff'] = zero_speed_df['TripTime'].diff().fillna(0)
                zero_speed_periods = zero_speed_df['time_diff'].cumsum()
                
                if (zero_speed_periods >= 100).any():
                    return os.path.basename(filepath)
    except Exception as e:
        print(f"Error processing file {filepath}: {e}")
    return None

def main():
    # Start a Dask client with a specified port
    client = Client(dashboard_address=':37868')  # You can choose any available port here

    # Get list of all CSV files in the directory
    csv_files = [os.path.join(directory, filename) for filename in os.listdir(directory) if filename.endswith(".csv")]

    # Create a Dask bag of the file paths
    file_bag = db.from_sequence(csv_files, npartitions=10)
    
    # Map the check_file function to the file paths
    results = file_bag.map(check_file).compute()

    # Filter out None results
    files_with_long_stops = [result for result in results if result]

    return files_with_long_stops

if __name__ == "__main__":
    result_files = main()
    print(result_files)


C:\Users\BSL\AppData\Roaming\Python\Python312\site-packages\distributed\node.py:182: UserWarning: Port 37868 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 14884 instead
  warnings.warn(
C:\Program Files\Python312\Lib\contextlib.py:144: UserWarning: Creating scratch directories is taking a surprisingly long time. (1.69s) This is often due to running workers on a network file system. Consider specifying a local-directory to point workers to write scratch data to a local disk.
  next(self.gen)
Task exception was never retrieved
future: <Task finished name='Task-113577' coro=<Client._gather.<locals>.wait() done, defined at C:\Users\BSL\AppData\Roaming\Python\Python312\site-packages\distributed\client.py:2197> exception=AllExit()>
Traceback (most recent call last):
  File "C:\Users\BSL\AppData\Roaming\Python\Python312\site-packages\distributed\client.py", line 2206, in wait
    raise AllExit()
distributed.client.AllExit


KeyboardInterrupt: 

Task exception was never retrieved
future: <Task finished name='Task-113580' coro=<Client._gather.<locals>.wait() done, defined at C:\Users\BSL\AppData\Roaming\Python\Python312\site-packages\distributed\client.py:2197> exception=AllExit()>
Traceback (most recent call last):
  File "C:\Users\BSL\AppData\Roaming\Python\Python312\site-packages\distributed\client.py", line 2206, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-113578' coro=<Client._gather.<locals>.wait() done, defined at C:\Users\BSL\AppData\Roaming\Python\Python312\site-packages\distributed\client.py:2197> exception=AllExit()>
Traceback (most recent call last):
  File "C:\Users\BSL\AppData\Roaming\Python\Python312\site-packages\distributed\client.py", line 2206, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-113581' coro=<Client._gather.<locals>.wait() done, defined at C:\